In [ ]:
# Standard Library Imports
import os
import re
import time
import multiprocessing

# Third Party Imports
import pprint
from glob import glob
import numpy as np
import webbpsf
from astropy.io import fits
from astropy.io import ascii as asc
import matplotlib.pyplot as plt

# Local Imports (from nircam_simulator package)
from mirage import imaging_simulator
from mirage.seed_image import catalog_seed_image
from mirage.dark import dark_prep
from mirage.ramp_generator import obs_generator
from mirage.apt import apt_inputs
from mirage.yaml import yaml_generator, write_observationlist
from mirage.catalogs import get_catalog
from mirage.psf import psf_library

# View matplotlib plots inline
%matplotlib inline

# First, generate (fake) PSF library files

Let's simulate Image Array 1 (OTE-06).
It has 6 observations:
- The first 5 observations are WFSC Commissioning observations, that each include 18 WFC groups. (18 sets of image-move-image. Right?)
- The last observation is simply NIRCam imaging to see how we did.

So I will need 5 x 18 = 90 distinct PSF library files. 

### Define location of APT files and catalogs

In [ ]:
prop_id = 1140

# Change if you put your files somewhere else
ote_dir = '/Users/lchambers/TEL/mirage/OTECommissioning/OTE06'

# Change if you named your files differently.
root = 'OTE06-{}_withPSFs'.format(prop_id)

pointing_file = os.path.join(ote_dir, 'OTE06-1140.pointing')
xml_file = os.path.join(ote_dir, 'OTE06-1140.xml')

In [ ]:
# Get SW and LW catalogs
cats = get_catalog.get_all_catalogs(pointing_file, prop_id)
target_coords, catalog_filenames_sw, catalog_filenames_lw = cats

### Get the observation/visit information from the APT files

In [ ]:
# Get the information from the pointing file
apt_prop = apt_inputs.AptInput()
pointing_tab = apt_prop.get_pointing_info(pointing_file, '1140')

In [ ]:
# Create dictionary that mirrors the program structure

program_structure = {}
for visit_id in pointing_tab['visit_id']:
    visit_num = visit_id[4:7]
    obs_num = visit_id[7:]
    
    obs_key = 'Observation{}'.format(obs_num)
    visit_key = 'Visit{}'.format(visit_num)
    
    program_structure.setdefault(obs_key, {})
    visit_dict = program_structure[obs_key].setdefault(visit_key, []).append('Exposure')                                                                      
    
# pprint.pprint(program_structure)  

In [ ]:
# Create directory structure based on dictionary
library_root_dir = '/Users/lchambers/TEL/mirage/test_library/'

program_dir = os.path.join(library_root_dir, 'OTE06-1140')
for observation in program_structure.keys():
    for visit in program_structure[observation].keys():
        visit_dir = os.path.join(program_dir, observation, visit)
        if not os.path.exists(visit_dir):
            os.makedirs(visit_dir)

## Generate the original PSF array

In [ ]:
# Generate original PSF

# Let's go for a global alignment-esque thing, with the image array and random pistons.
nc = webbpsf.NIRCam()
nc, ote = webbpsf.enable_adjustable_ote(nc)

# Set up the array
webbpsf.opds.setup_image_array(ote, reset=True, verbose=False, size='large', )

# Add random pistons
random_pistons = np.random.randn(18)*500  # substantial coarse phasing erorrs. 
for i, seg in enumerate(ote.segnames[0:18]):  # don't piston "segment 19" the SM
    ote.move_seg_local(seg, piston=random_pistons[i])


In [ ]:
original_psf = nc.calc_psf(monochromatic=2e-6, oversample=1, fov_pixels=2048, display=True)

## Make slightly different PSFs for every exposure

In [ ]:
def create_lib_for_random_ote(i):
    start_time = time.time()
    obs = 'Observation{}'.format(pointing_tab['obs_num'][i])
    visit = 'Visit{}'.format(pointing_tab['visit_num'][i])
    visit_dir = os.path.join(program_dir, obs, visit)
    
    print('Calculating PSF for obs {}, visit {}, exposure {}'
          .format(pointing_tab['obs_num'][i], pointing_tab['visit_num'][i], i + 1))

    i_seg = random_segments[i]
    ote.move_seg_local(ote.segnames[i_seg], piston = random_pistons[i])

    library_filename = 'nircam_f212n_fovp2048_samp1_npsf1_exp{}.fits'.format(i + 1)
    c = psf_library.CreatePSFLibrary('NIRCam', filters='F212N', detectors='NRCA3', 
                                     fov_pixels=2048, oversample=1, num_psfs=1, 
                                     fileloc=visit_dir, filename=library_filename,
                                     pupil_opd=ote)
    c.create_files()
    
    print('Elapsed time: {}\n'.format(time.time() - start_time))

In [ ]:
# Define a beep noise
beep = lambda x: os.system("echo -n '\a';sleep 0.2;" * x)

In [ ]:
# Determine how many cores I have
multiprocessing.cpu_count()

In [ ]:
# Generate PSF library files for each exposure:
n_exposures = len(pointing_tab['visit_id'])

random_segments = np.random.randint(0, 17, n_exposures)
random_pistons = np.random.randn(n_exposures)*500

# for i in range(n_exposures):
#     create_lib_for_random_ote(i)

# p = Pool(8)
# p.map(create_lib_for_random_ote, range(n_exposures))
create_lib_for_random_ote(0)
    
beep(10)

In [ ]:
# Generate PSF library files!
hdu = fits.PrimaryHDU()
hdu.data = ote.opd
hdu.header['BUNIT'] = 'meter'
opd_fits = fits.HDUList(hdu)

c = psf_library.CreatePSFLibrary('NIRCam', filters='F212N', detectors='NRCA3', 
                                 fov_pixels=2048, oversample=1, num_psfs=1, 
                                 fileloc=library_root_dir, pupil_opd=opd_fits)
c.create_files()

In [ ]:
p = Pool(8)
p.map(create_lib_for_random_ote, range(n_exposures))